# Changes

run_command_line.py => 

in main() 

parser.add_argument('--image',default='c2.png')
parser.add_argument('--config',default='configuration.ini')

checkpoint_path = '../east_icdar2015_resnet_v1_50_rbox'
AND create that path

text_recognition.py => 

by PyTessBaseAPI()


PyTessBaseAPI(path='/usr/share/tesseract-ocr/4.00/tessdata',psm=PSM.AUTO_OSD, oem=OEM.LSTM_ONLY) as api:

In [0]:
!rm -r strabo-text-recognition-deep-learning/
#!pip install gdown
!gdown https://drive.google.com/uc?id=1MjBMO_Ql7kth7VPPX6XkDXW4sr1qtjDV
!gdown https://drive.google.com/open?id=17Tat38W_M_4yaHS_dHTf2TakNaT-jQel
!sh install.sh
!sudo apt-get install python3-dev
!sudo apt-get install software-properties-common
!sudo apt-get install libgeos-dev
!sudo pip3 install -r text-detection-requirements.txt
!sudo add-apt-repository -y ppa:alex-p/tesseract-ocr
!sudo apt-get update
!sudo apt install -y libtesseract-dev libleptonica-dev liblept5
!sudo apt install -y tesseract-ocr
!sudo pip3 install git+https://github.com/spatial-computing/tesserocr
!export LC_ALL=C
%cd strabo-text-recognition-deep-learning/

In [0]:
targetFile = "pray"
# you can but prefix with a $
targetFileImage = targetFile + ".png"
targetFileGeo = targetFile + ".tfw"
targetFileMeta = targetFile + ".prj"
# !python3 run_command_line.py --image=$targetFileImage

In [124]:
#Make Changes From Above Now
from math import radians, cos, sin, asin, sqrt,pi
def transformLatLonByXMetres(lat,lon,dn,de):
    R=6377563.396 #from their files
    dLat = dn/R
    dLon = de/(R*cos(pi*lat/180))
    latO = lat + dLat * 180/pi
    lonO = lon + dLon * 180/pi

    return (latO,lonO)

(53.80695614038061, -2.2333605268579597)


In [128]:
## Strabo Infrastructure
import json 
import shutil
import os 

def getBaseLatLon():
  return 53.9153862, -2.6224470139

def convertPixelToMetreOffset(coord):
  pixelX, pixelY = coord
  with open(targetFileGeo, 'rb') as f:
    lines = f.readlines()
    lonOffsetFromBase = float(lines[-1].strip())
    latOffsetFromBase = float(lines[-2].strip())
    yPixelsToMetres = 1 / float(lines[-3].strip())
    xPixelsToMetres = 1 / float(lines[0].strip())

    metreOffsetOnY = pixelY / yPixelsToMetres
    metreOffsetOnX = pixelX / xPixelsToMetres

    return (metreOffsetOnX+latOffsetFromBase,metreOffsetOnY+lonOffsetFromBase)

def mapImagePixelToLatLon(coord):
  lat,lon = getBaseLatLon()
  Xm, Ym = convertPixelToMetreOffset(coord)
  newLatFromBase, newLonFromBase = transformLatLonByXMetres(lat,lon,Ym,Xm)
  return newLatFromBase, newLonFromBase

def mapBboxToSingleCoord(bboxCoords):
  xs = list(map(lambda x: x[0], bboxCoords))
  ys = list(map(lambda x: x[1], bboxCoords))

  avX = (min(xs) + max(xs)) / 2
  avY = (min(ys) + max(ys)) / 2

  return (avX,avY)

def isChimney(proposalString):
  return "Chimney" in proposalString

def locateChimneyPixelsInStraboJson(finalJson):
  featureList = finalJson["features"]
  featuresOfInterest = list(filter(lambda x: isChimney(x["NameAfterDictionary"]),featureList))
  bboxes = []
  for entry in featuresOfInterest:
    uniqueCoords = (list(set(list(map(lambda x: (x[0],x[1]),entry['geometry']['coordinates'][0])))))
    if len(uniqueCoords) == 4:
      bboxes.append(uniqueCoords)
  
  return bboxes

def retrieveModelOutputStrabo():
  basePath = "static/results/"
  topLevelName =  basePath+os.listdir(basePath)[0]+"/"
  resultBase = topLevelName+list(filter(lambda x: os.path.isdir(topLevelName+x),os.listdir(topLevelName)))[0]
  with open(resultBase+"/final.txt",'rb') as f:
    parsed = (json.load(f))
  # shutil.rmtree(topLevelName)
  bboxChimneyLocations = (locateChimneyPixelsInStraboJson(parsed))
  singleXY = list(map(lambda x: mapBboxToSingleCoord(x),bboxChimneyLocations))
  latLonChimneyPoints = list(map(lambda x: mapImagePixelToLatLon(x),singleXY))
  print(latLonChimneyPoints)

def retrieveChimneyLatLonsFromImage():
  chimneyPixels = retrieveModelOutputStrabo()

retrieveChimneyLatLonsFromImage()

[(53.78968024848036, -2.2709771291072096), (53.789568238182945, -2.2709868190952593), (53.78979644963923, -2.2709415991510276), (53.78926630566357, -2.270738755401188)]


In [65]:
retrieveChimneyLatLonsFromImage()

<generator object walk at 0x7f1289d4edb0>


In [63]:
retrieveChimneyLatLonsFromImage()

TypeError: ignored

In [35]:
!python3 run_command_line.py  --image="pray.png"

../imgs/pray.png
defaultdict(<class 'int'>, {'result_path': 'static/results/test2'})
image name is ../imgs/pray.png
/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)
make: Entering directory '/content/strabo-text-recognition-deep-learning/lanms'
make: 'adaptor.so' is up to date.
make: Leaving directory '/content/strabo-text-recognition-deep-learning/lanms'
resnet_v1_50/block1 (?, ?, ?, 256)
resnet_v1_50/block2 (?, ?, ?, 512)
resnet_v1_50/block3 (?, ?, ?, 1024)
resnet_v1_50/block4 (?, ?, ?, 2048)
Shape of f_0 (?, ?, ?, 2048)
Shape of f_1 (?, ?, ?, 512)
Shape of f_2 (?, ?, ?, 256)
Shape of f_3 (?, ?, ?, 64)
Shape of h_0 (?, ?, ?, 2048), g_0 (?, ?, ?, 2048)
Shape of h_1 (?, ?, ?, 128), g_1 (?, ?, ?, 128)
Shape of h_2 (?, ?, ?, 64), g_2 (?, ?, ?, 64)
Shape of h_3 (?, ?, ?, 32), g_3 (?, ?, ?, 32)
2020-04-20 17:33:18.809463: I tensorflow/